In [1]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 500, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
params = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [2]:
import pandas as pd
data= pd.read_csv('../ignore/data.csv').drop(columns=['x','y','z'])
data.head()

,carat,cut,color,clarity,depth,table,price
0,2.26,Ideal,G,SI2,61.9,57.0,12831
1,2.43,Very Good,H,SI2,63.2,57.0,16170
2,0.80,Premium,F,SI2,61.0,57.0,2797
3,0.40,Ideal,F,I1,63.3,60.0,630
4,0.31,Ideal,G,VS2,61.6,55.0,698


In [3]:
data.columns

Index(['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price'], dtype='object')

In [3]:
data.price

count    40455.000000
mean      3939.242813
std       4000.344155
min        326.000000
25%        950.000000
50%       2409.000000
75%       5331.000000
max      18823.000000
Name: price, dtype: float64

In [4]:
from pca import convertToNumber
nDf= convertToNumber(data[['cut','color','clarity']])

cut converted
color converted
clarity converted


/home/curro/repos/kaggle-comp/code/pca.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataObj[col]=getPCAoneCom(dataObj[col]).astype('float32')
/home/curro/repos/kaggle-comp/code/pca.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataObj[col]=getPCAoneCom(dataObj[col]).astype('float32')
/home/curro/repos/kaggle-comp/code/pca.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [5]:
nDf.head()

,cut,color,clarity
0,0.690766,0.840422,-0.084403
1,-0.431324,-0.161861,-0.084403
2,-0.611382,-0.325315,-0.084403
3,0.690766,-0.325315,-0.024296
4,0.690766,0.840422,-0.657709


In [6]:
for col in nDf.columns:
    data[col]=nDf[col]

In [7]:
data.shape

(40455, 7)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns='price'), data.price, test_size=0.2)

In [ ]:
from dask.distributed import Client, progress
import joblib
from sklearn.ensemble import RandomForestRegressor

client = Client(n_workers=4, threads_per_worker=4, memory_limit='11GB')
bestMod={'model': RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=200,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False), 'R**2': 0.9737698838998453}
print(client)
with joblib.parallel_backend('dask'):
    for estimators in params['n_estimators']:
        for features in params['max_features']:
            for dep in params['max_depth']:
                for samples in params['min_samples_split']:
                    for samplesL in params['min_samples_leaf']:
                        for boot in params['bootstrap']:
                            model=RandomForestRegressor(n_estimators=estimators,max_features=features,
                            max_depth=dep,min_samples_split=samples,min_samples_leaf=samplesL,
                            bootstrap=boot)
                            model.fit(X_train,y_train)
                            if model.score(X_test, y_test)>bestMod['R**2']:
                                bestMod={'model':model,'R**2':model.score(X_test, y_test)}
                                print({'model':model,'R**2':model.score(X_test, y_test)})
                                
                            del model

<Client: 'tcp://127.0.0.1:42067' processes=4 threads=16, memory=44.00 GB>
{'model': RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=200,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False), 'R**2': 0.9748184894201276}
{'model': RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=2, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=200,
                      n_jobs=None, oob_score=False, random_state=None,
    

In [14]:
from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=30,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=2, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=200,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

model.fit(X_train, y_train)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=30,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=2, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=200,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [15]:
print({'model':model,'R**2':model.score(X_test, y_test)})

{'model': RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=30,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=2, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=200,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False), 'R**2': 0.9786792646539269}


In [16]:
pred=model.predict(X_test)
df=pd.DataFrame(y_test)
df['prediccion']=pred
display(df)

,price,prediccion
4212,6071,6255.834673
33935,14199,14108.354280
20854,928,778.570847
37743,698,684.455139
8733,3398,3628.027990
...,...,...
31613,2327,3228.970982
39045,1435,1466.968774
36365,7761,7926.161268
11268,436,438.817631


### paso

In [1]:
import pandas as pd
data= pd.read_csv('../ignore/data.csv').drop(columns=['x','y','z'])
data.head()
data.dtypes

carat      float64
cut         object
color       object
clarity     object
depth      float64
table      float64
price        int64
dtype: object

In [2]:
from pca import convertToNumber
nDf= convertToNumber(data[['cut','color','clarity']])

cut converted
color converted
clarity converted


/home/curro/repos/kaggle-comp/code/pca.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataObj[col]=getPCAoneCom(dataObj[col]).astype('float32')
/home/curro/repos/kaggle-comp/code/pca.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataObj[col]=getPCAoneCom(dataObj[col]).astype('float32')
/home/curro/repos/kaggle-comp/code/pca.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [3]:
for col in nDf.columns:
    data[col]=nDf[col]

In [4]:
#data.price=data.price.apply(lambda p:p/10**5)
data.price.describe()

count    40455.000000
mean      3939.242813
std       4000.344155
min        326.000000
25%        950.000000
50%       2409.000000
75%       5331.000000
max      18823.000000
Name: price, dtype: float64

In [5]:
data.price=data.price.apply(lambda num:num/10**5)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns='price'), data.price, test_size=0.05)

In [7]:
X_train.columns

Index(['carat', 'cut', 'color', 'clarity', 'depth', 'table'], dtype='object')

In [8]:
from sklearn.ensemble import RandomForestRegressor
model= RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=110,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=10,
                      min_weight_fraction_leaf=0.0, n_estimators=333,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=110,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=10,
                      min_weight_fraction_leaf=0.0, n_estimators=333,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [9]:
print({'model':model,'R**2':model.score(X_test, y_test)})

{'model': RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=110,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=10,
                      min_weight_fraction_leaf=0.0, n_estimators=333,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False), 'R**2': 0.9794699830426955}


In [10]:
test= pd.read_csv('../ignore/test.csv').drop(columns=['x','y','z'])
test.head()

,id,carat,cut,color,clarity,depth,table
0,0,0.56,Ideal,I,VS2,62.1,54.0
1,1,1.05,Ideal,G,VS2,61.9,56.0
2,2,0.50,Premium,E,VS2,61.5,56.0
3,3,0.80,Ideal,F,VS1,62.8,56.0
4,4,0.54,Ideal,G,VS1,61.4,57.0


In [11]:
nDf= convertToNumber(test[['cut','color','clarity']])

cut converted
color converted
clarity converted


/home/curro/repos/kaggle-comp/code/pca.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataObj[col]=getPCAoneCom(dataObj[col]).astype('float32')
/home/curro/repos/kaggle-comp/code/pca.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataObj[col]=getPCAoneCom(dataObj[col]).astype('float32')
/home/curro/repos/kaggle-comp/code/pca.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [12]:
for col in nDf.columns:
    test[col]=nDf[col]

In [13]:
price=model.predict(test[['carat', 'cut', 'color', 'clarity', 'depth', 'table']])

In [14]:
subir=pd.DataFrame(price)
subir.columns=['price']
subir.index.name='id'
subir.price=subir.price.apply(lambda p:int(round(p*10**5,0)))
subir.head()

,price
id,
0,1299
1,6581
2,1695
3,3975
4,1756


In [15]:
subir.price=subir.price.apply(lambda p:int(round(p,0)))
subir.head()

,price
id,
0,1299
1,6581
2,1695
3,3975
4,1756


In [16]:
subir.to_csv('../ignore/sample_submission.csv')

In [27]:
submision=pd.read_csv('../ignore/sample_submission.csv')
submision.head()

,Unnamed: 0,id,price
0,0,0,1247
1,1,1,6580
2,2,2,1684
3,3,3,3882
4,4,4,1832
